# Exploração dos dados de candidatos

[Voltar ao Índice](00_indice.ipynb)

Vamos verificar as características dos dados dos candidatos, quando completos com todas as informações vindas de todas as fontes (e.g. Bens, eleitorado, quantidade de votos). Especificamente, queremos verificar por que alguns candidatos não possuem votos registrados.

In [2]:
import src.analysis_functions as af
import src.xavy.data_retriever as xz

## Carregando os dados

In [4]:
# Baixa os dados caso não estejam presentes:
xz.sync_remote_zipped_files('https://nuvem.ceweb.br/s/2HGbHTrWmnLwqjr/download/consulta_cand_2020_BRASIL.zip', '../dados/limpos/tse/consulta_cand/', keep_zip_dir=False)
xz.sync_remote_zipped_files('https://nuvem.ceweb.br/s/NkEp3KoDp2HP8dc/download/perfil_eleitorado_2022.zip', '../dados/limpos/tse/perfil_eleitorado/', keep_zip_dir=False)
xz.sync_remote_zipped_files('https://nuvem.ceweb.br/s/xq6nGLHsEZpgMHo/download/bem_candidato_2022_BRASIL.zip', '../dados/limpos/tse/bens_candidatos/', keep_zip_dir=False)
xz.sync_remote_zipped_files('https://nuvem.ceweb.br/s/2SpdZPeMaDtdRAf/download/votacao_candidato_munzona_2022.zip', '../dados/brutos/tse/', keep_zip_dir=False)

# Carrega informações sobre o candidato, seus bens e sobre o número de eleitores na sua unidade eleitoral:
cand_df = af.load_cand_eleitorado_bens_votos('../dados/limpos/tse/consulta_cand/consulta_cand_2022_BRASIL.csv', # '../dados/limpos/tse/consulta_cand/consulta_cand_2022_BRASIL_pos.csv',
                                             '../dados/limpos/tse/perfil_eleitorado/perfil_eleitorado_2022.csv',
                                             '../dados/limpos/tse/bens_candidatos/bem_candidato_2022_BRASIL.csv',
                                             '../dados/brutos/tse/votacao_candidato_munzona_2022/votacao_candidato_munzona_2022_BRASIL.csv',
                                             cand_sel_cols=['SQ_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO', 'NR_CPF_CANDIDATO', 'NR_TITULO_ELEITORAL_CANDIDATO', 'SG_PARTIDO', 'SG_UF', 
                                                            'SG_UE', 'NM_UE', 'DS_CARGO', 'NM_SOCIAL_CANDIDATO', 'NR_IDADE_DATA_POSSE', 'DS_GENERO', 'DS_GRAU_INSTRUCAO', 'DS_COR_RACA',
                                                            'DS_SITUACAO_CANDIDATURA', 'DS_DETALHE_SITUACAO_CAND'])

Found a local file, skip download.
Found a local file, skip download.
Found a local file, skip download.
Found a local file, skip download.


## Candidatos sem votos

### Suplentes e vices recebem votos?

In [5]:
cargos_vice = ['1º SUPLENTE', '2º SUPLENTE', 'VICE-GOVERNADOR', 'VICE-PRESIDENTE']
vice_df = cand_df.loc[cand_df['DS_CARGO'].isin(cargos_vice)]

In [6]:
vice_df['QT_VOTOS_NOMINAIS'].value_counts(dropna=False)

NaN    709
Name: QT_VOTOS_NOMINAIS, dtype: int64

**Resultado:** Se temos apenas ausência de contabilização (NaN) para os suplentes e vices (veja acima), então eles não recebem votos

### E os demais cargos que não tem votos?

In [7]:
no_votes_df = cand_df.loc[cand_df['QT_VOTOS_NOMINAIS'].isnull() & ~cand_df['DS_CARGO'].isin(cargos_vice)]

In [8]:
no_votes_df['DS_DETALHE_SITUACAO_CAND'].value_counts()

AGUARDANDO JULGAMENTO    1964
RENÚNCIA                   27
DEFERIDO                    1
Name: DS_DETALHE_SITUACAO_CAND, dtype: int64

In [9]:
no_votes_df['DS_SITUACAO_CANDIDATURA'].value_counts()

CADASTRADO    1964
INAPTO          27
APTO             1
Name: DS_SITUACAO_CANDIDATURA, dtype: int64

**Resultado:** Vemos que a maioria dos casos de candidatos sem votos são de pessoas vetadas.

Vamos investigar os casos de candidatos aptos mas que não receberam votos

In [10]:
aptos_no_votes_df = no_votes_df.loc[no_votes_df['DS_SITUACAO_CANDIDATURA'] == 'APTO']